In [1]:
# Compound + Complexity 배경지식 확인

from openai import OpenAI
api_key = '### YOUR OPENAI API KEY ###'

client = OpenAI(api_key = api_key)

# upload trian data
# not a sentence나 simple sentence가 높으면 낮음
# compound + complexity가 연속되지 않고 적절히 섞여 있으면 높음
client.files.create(
  file=open("### your path ### /sentence_complexity_sample.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-6jffQldteA0RsxnqsxjJoK0h', bytes=8808, created_at=1709650797, filename='sentence_complexity_sample.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [2]:
# Fine-tuning
client.fine_tuning.jobs.create(training_file="### FileObject id ###", model="gpt-3.5-turbo")

FineTuningJob(id='ftjob-pPGHmjmKjWYm88uJpmmiEoZb', created_at=1709650814, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-MRfWIXUVyw7BOHgCC1StLwLu', result_files=[], status='validating_files', trained_tokens=None, training_file='file-6jffQldteA0RsxnqsxjJoK0h', validation_file=None, user_provided_suffix=None)

In [5]:
client.fine_tuning.jobs.retrieve("### FineTuningJob id ###")

FineTuningJob(id='ftjob-pPGHmjmKjWYm88uJpmmiEoZb', created_at=1709650814, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::8zQfetoN', finished_at=1709651221, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-MRfWIXUVyw7BOHgCC1StLwLu', result_files=['file-uPf7jcob8bRqyGAgD559y1s8'], status='succeeded', trained_tokens=20870, training_file='file-6jffQldteA0RsxnqsxjJoK0h', validation_file=None, user_provided_suffix=None)

In [6]:
response = client.chat.completions.create(
    model = "### fine_tuned_model ###",
    messages = [
        {"role": "system", "content": "스크립트 내에 Coumpound, Complexity, Simple Sentence(하나의 independent 문장), Not a sentence(완전한 문장이 아닌 경우)를 각각 구해서 갯수를 출력하고 구성을 평가"},
        {"role": "user", "content": "Well, Korea has many mountains, rivers, and everything in between. The eastern part of Korea is higher than the western part. We call this topography “Donggo Seojeo” in Korean language. Personally, I prefer the east because the color of the sea is more beautiful there than in the west. And we have Jeju Island, which is the most beautiful and famous travel destination in Korea. It’s located at the southernmost part of Korea, so the temperature is higher and warmer than other parts. That’s why it’s a perfect destination for vacation... Hmm... I think you have learned all about the geography of Korea."}
    ]
)

In [7]:
import json

res_json = json.loads(response.json())
res_json["choices"][0]["message"]["content"]

'- Coumpound: 1개 (Well, Korea has many mountains, rivers, and everything in between.)\n- Complexity: 2개 (The eastern part of Korea is higher than the western part. / Personally, I prefer the east because the color of the sea is more beautiful there than in the west.)\n- Simple Sentence: 1개 (And we have Jeju Island, which is the most beautiful and famous travel destination in Korea.)\n- Not a sentence: 1개 (Hmm... I think you have learned all about the geography of Korea.)\n\n피드백: 다양한 구성이 사용되었고, 적절한 비율로 문장들이 활용되었습니다. Coumpound 는 한 문장 내에서 쉼표 등으로 연결된 구문, Complexity는 여러 개념이 섞인 구문, Simple한 문장은 하나의 완전한 개념이 포함된 문장, Not a sentence는 완전한 문장이 아닌 경우(불완전한 문장, 감탄사 등)'

In [10]:
response = client.chat.completions.create(
    model = "### fine_tuned_model ###",
    messages = [
        {"role": "system", "content": "스크립트 내에 Coumpound, Complexity, Simple Sentence(하나의 independent 문장), Not a sentence(완전한 문장이 아닌 경우)를 각각 구해서 갯수를 출력하고 구성을 평가"},
        {"role": "user", "content": "Hi, I’m calling to ask some questions about your MP3 player. Actually, I need to borrow an MP3 player for this coming Sunday. So um... Do you still have the MP3 player that you showed me last month? Then, are you planning on using it this Sunday, by any chance? Oh, you won’t use it. so... Can I borrow it for just one day then? Thank you. I promise to take good care of it. And when would you like me to return it to you? Sure, I’ll make sure to return it on time."}
    ]
)

In [11]:
import json

res_json = json.loads(response.json())
print(res_json["choices"][0]["message"]["content"])

'- Compound: 15\n- Complexity: 1\n- Simple Sentence: 3\n- Not a sentence: 0'